# 환경설정

In [7]:
# 설치(import)
import os
import torch
import gc
import re
from PIL import Image
from transformers import AutoProcessor, LlavaOnevisionForConditionalGeneration, BitsAndBytesConfig
from tqdm.notebook import tqdm # 노트북용 진행 바 추가

# [필수] 메모리 파편화 방지 및 관리 설정
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

def clean_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

clean_memory()
print("준비 완료!")

준비 완료!


# VARCO VISION 2.0

In [8]:
# 기존에 모델이 메모리에 있다면 삭제 시도 (재실행 시 안전장치)
if 'model' in locals():
    del model
    clean_memory()

model_id = "NCSOFT/VARCO-VISION-2.0-1.7B-OCR"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

print("모델 로딩 중 (VRAM 절약 모드)...")
model = LlavaOnevisionForConditionalGeneration.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation="sdpa",
    low_cpu_mem_usage=True
)
processor = AutoProcessor.from_pretrained(model_id)
print("모델 로드 완료!")

모델 로딩 중 (VRAM 절약 모드)...
모델 로드 완료!


In [9]:
def clean_ocr_text(text):
    # 좌표(소수점 4개 세트) 제거
    cleaned = re.sub(r'\d+\.\d+,\s*\d+\.\d+,\s*\d+\.\d+,\s*\d+\.\d+', '', text)
    # 불필요한 공백 및 줄바꿈 정리
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()
    return cleaned

In [10]:
# 1. 경로 설정 (노트북 파일과 같은 위치에 있는 'img' 폴더 기준)
input_folder = "img" 
image_extensions = (".png", ".jpg", ".jpeg", ".bmp")

if not os.path.exists(input_folder):
    print(f"경고: '{input_folder}' 폴더가 존재하지 않습니다.")
else:
    image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(image_extensions)]
    print(f"총 {len(image_files)}개의 이미지를 발견했습니다.")

    # 2. 반복문 실행 (tqdm으로 진행률 표시)
    for filename in tqdm(image_files, desc="OCR 처리 중"):
        image_path = os.path.join(input_folder, filename)
        
        try:
            image = Image.open(image_path).convert("RGB")
            
            # 6GB VRAM 안전 사이즈
            target_size = 512 
            w, h = image.size
            if max(w, h) > target_size:
                scale = target_size / max(w, h)
                image = image.resize((int(w * scale), int(h * scale)), resample=Image.LANCZOS)

            conversation = [
                {"role": "user", "content": [{"type": "image", "image": image}, {"type": "text", "text": "<ocr>"}]}
            ]

            inputs = processor.apply_chat_template(
                conversation, add_generation_prompt=True, tokenize=True, return_dict=True, return_tensors="pt"
            ).to(model.device)

            with torch.inference_mode():
                generate_ids = model.generate(
                    **inputs,
                    max_new_tokens=1024,
                    do_sample=False,
                    use_cache=True
                )

            full_output = processor.decode(generate_ids[0][len(inputs.input_ids[0]):], skip_special_tokens=True)
            print(f"[{filename}] 결과: {clean_ocr_text(full_output)}")

            # 메모리 점유 방지
            del inputs, generate_ids, image
            clean_memory()

        except Exception as e:
            print(f"에러 발생 ({filename}): {e}")
            clean_memory()

    print("\n모든 이미지 처리가 완료되었습니다!")

총 7개의 이미지를 발견했습니다.


OCR 처리 중:   0%|          | 0/7 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


[Reimg1.png] 결과: 20XXX년 5월 8일 수요일 날씨 맑음 오늘은 어버이날이 다. 부모님께 카네이 선을 만들어서 드렸다 고맙다고 하섰다


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


KeyboardInterrupt: 